# RISC-V Baseline Jupyter Notebook

### Import Libraries

In [1]:
# Import Libraries
from pynq import Overlay
from time import sleep
from pynq.overlays.base import BaseOverlay
from pynq.lib import Pmod_ADC
%matplotlib inline
import matplotlib.pyplot as plt
import struct

### Import Overlay / Set Up Signals

In [3]:
# Load RISCV2Core Processor
rv = Overlay("RV2Core.bit")
# Define Input Signal (AXI GPIO)
rstN = rv.rstN
step = rv.step
runAll = rv.runAll
hostCtrl = rv.hostCtrl
host_memWr = rv.host_memWr
host_memAdd = rv.host_memAdd
host_datToMem = rv.host_datToMem
# Define Output Signal (AXI GPIO)
datToHost = rv.datToHost
# Define Register Banks
ABI_Array = ['zero','ra','sp','gp','tp','t0','t1','t2','s0/fp','s1','a0','a1','a2','a3','a4','a5','a6','a7',
             's2','s3','s4','s5','s6','s7','s8','s9','s10','s11','t3','t4','t5','t6']

### Project Set Up

In [4]:
# This block will contain all the neccessary flags and variables

# Print Verbose - Print all information
verbose_prints = True

# Paths to Machine Code
IM0_filename = "asm_examples/swapX3DigitX4WithX5DigitX6.rv"
IM1_filename = "asm_examples/blank.txt"

# Number of Steps to Run
number_of_steps = 300

# Period
period = 2e-08 #20ns

### Read Instruction Memory

In [19]:
IM0Dat = []
IM1Dat = []

for RVCore in range(0,2):
    hostCtrl.write(0,int('1',2))
    host_memWr.write(0,int('0',2))
    bit = str(RVCore)
    host_memAdd_31_16 = int('0000000000000000',2)
    host_memAdd_15 = int(bit,2)
    host_memAdd_14_13 = int('00',2) #binary
    host_memAdd_12_6 = int('0000000',2) #binary
    host_memAdd_5_0 = int('000000',2) #binary
    host_memAdd_str = '{0:016b}'.format(host_memAdd_31_16) + '{0:01b}'.format(host_memAdd_15) + '{0:02b}'.format(host_memAdd_14_13) + '{0:07b}'.format(host_memAdd_12_6) + '{0:06b}'.format(host_memAdd_5_0)
    if RVCore == 0:
        print("Instruction Memory of RVCore0\n")
    elif RVCore == 1:
        print("\nInstruction Memory of RVCore1\n")
    for i in range(0,64):
        host_memAdd_5_0 = int('{0:06b}'.format(i),2)
        host_memAdd_str = '{0:016b}'.format(host_memAdd_31_16) + '{0:01b}'.format(host_memAdd_15) + '{0:02b}'.format(host_memAdd_14_13) + '{0:07b}'.format(host_memAdd_12_6) + '{0:06b}'.format(host_memAdd_5_0)
        host_memAdd.write(0,int(host_memAdd_str,2))
        if RVCore == 0:
            IM0Dat.append(hex(datToHost.read()))
        elif RVCore == 1:
            IM1Dat.append(hex(datToHost.read()))
        if verbose_prints:
            if(hex(datToHost.read()) != "0x0"):
                print("Host Address:", hex(i), "\tData: ", hex(datToHost.read()))
        elif verbose_prints:
            print("Host Address:", hex(i), "\tData: ", hex(datToHost.read()))
        sleep(period) 
    hostCtrl.write(0,int('0',2))
    host_memAdd_15 = int('0',2) 
    host_memAdd_5_0 = int('000000',2) 
    host_memAdd_str = '{0:016b}'.format(host_memAdd_31_16) + '{0:01b}'.format(host_memAdd_15) + '{0:02b}'.format(host_memAdd_14_13) + '{0:07b}'.format(host_memAdd_12_6) + '{0:06b}'.format(host_memAdd_5_0)
    host_memAdd.write(0,int(host_memAdd_str,2))
    sleep(10*period)
    
print("\nOperation Finished")

Instruction Memory of RVCore0

Host Address: 0x0 	Data:  0xd8000ef
Host Address: 0x1 	Data:  0x14000ef
Host Address: 0x2 	Data:  0x34000ef
Host Address: 0x3 	Data:  0x54000ef
Host Address: 0x4 	Data:  0x13
Host Address: 0x5 	Data:  0xffdff06f
Host Address: 0x6 	Data:  0xf00393
Host Address: 0x7 	Data:  0x413
Host Address: 0x8 	Data:  0x440863
Host Address: 0x9 	Data:  0x439393
Host Address: 0xa 	Data:  0x140413
Host Address: 0xb 	Data:  0xfe441ce3
Host Address: 0xc 	Data:  0x33f3b3
Host Address: 0xd 	Data:  0x33c1b3
Host Address: 0xe 	Data:  0x8067
Host Address: 0xf 	Data:  0xf00493
Host Address: 0x10 	Data:  0x513
Host Address: 0x11 	Data:  0x650863
Host Address: 0x12 	Data:  0x449493
Host Address: 0x13 	Data:  0x150513
Host Address: 0x14 	Data:  0xfe651ce3
Host Address: 0x15 	Data:  0x54f4b3
Host Address: 0x16 	Data:  0x54c2b3
Host Address: 0x17 	Data:  0x8067
Host Address: 0x18 	Data:  0x620663
Host Address: 0x19 	Data:  0x624a63
Host Address: 0x1a 	Data:  0x4625063
Host Address: 0x

### Read Memory Bank

In [12]:
MEM0Dat = []
MEM1Dat = []

for RVCore in range(0,2):
    hostCtrl.write(0,int('1',2)) 
    host_memWr.write(0,int('0',2)) 
    bit = str(RVCore)
    host_memAdd_31_16 = int('0000000000000000',2)
    host_memAdd_15 = int(bit,2)
    host_memAdd_14_13 = int('01',2) #binary
    host_memAdd_12_7 = int('000000',2) #binary
    host_memAdd_6_0 = int('0000000',2) #binary
    host_memAdd_str = '{0:016b}'.format(host_memAdd_31_16) + '{0:01b}'.format(host_memAdd_15) + '{0:02b}'.format(host_memAdd_14_13) + '{0:06b}'.format(host_memAdd_12_7) + '{0:07b}'.format(host_memAdd_6_0)
    if RVCore == 0:
        print("Memory of RVCore0\n")
    elif RVCore == 1:
        print("\nMemory of RVCore1\n")
    for i in range(0,68):
        host_memAdd_6_0 = int('{0:07b}'.format(i),2)
        host_memAdd_str = '{0:016b}'.format(host_memAdd_31_16) + '{0:01b}'.format(host_memAdd_15) + '{0:02b}'.format(host_memAdd_14_13) + '{0:06b}'.format(host_memAdd_12_7) + '{0:07b}'.format(host_memAdd_6_0)
        host_memAdd.write(0,int(host_memAdd_str,2))
        if RVCore == 0:
            MEM0Dat.append(hex(datToHost.read()))
        elif RVCore == 1:
            MEM1Dat.append(hex(datToHost.read()))
        if verbose_prints:
            if(hex(datToHost.read()) != "0x0"):
                print("Host Address:", hex(i), "\tData: ", hex(datToHost.read()))
        elif verbose_prints:
            print("Host Address:", hex(i), "\tData: ", hex(datToHost.read()))
        sleep(period)
    hostCtrl.write(0,int('0',2))
    host_memAdd_15 = int('0',2)
    host_memAdd_6_0 = int('0000000',2)
    host_memAdd_str = '{0:016b}'.format(host_memAdd_31_16) + '{0:01b}'.format(host_memAdd_15) + '{0:02b}'.format(host_memAdd_14_13) + '{0:06b}'.format(host_memAdd_12_7) + '{0:07b}'.format(host_memAdd_6_0)
    host_memAdd.write(0,int(host_memAdd_str,2))
    sleep(10*period)
    
print("\nOperation Finished")

Memory of RVCore0


Memory of RVCore1


Operation Finished


### Read Register Bank

In [18]:
RB0Dat = []
RB1Dat = []

for RVCore in range(0,2):
    hostCtrl.write(0,int('1',2))
    host_memWr.write(0,int('0',2))
    bit = str(RVCore)
    host_memAdd_15 = int(bit,2)
    host_memAdd_14_13 = int('10',2) #binary
    host_memAdd_12_5 = int('00000000',2) #binary
    host_memAdd_4_0 = int('00000',2) #binary
    host_memAdd_str = '{0:016b}'.format(host_memAdd_31_16) + '{0:01b}'.format(host_memAdd_15) + '{0:02b}'.format(host_memAdd_14_13) + '{0:08b}'.format(host_memAdd_12_5) + '{0:05b}'.format(host_memAdd_4_0)
    if RVCore == 0:
        print("Register Bank of RVCore0\n")
    elif RVCore == 1:
        print("\nRegister Bank of RVCore1\n")
    for i in range(0,32):
        host_memAdd_4_0 = int('{0:05b}'.format(i),2)
        host_memAdd_str = '{0:016b}'.format(host_memAdd_31_16) + '{0:01b}'.format(host_memAdd_15) + '{0:02b}'.format(host_memAdd_14_13) + '{0:08b}'.format(host_memAdd_12_5) + '{0:05b}'.format(host_memAdd_4_0)
        host_memAdd.write(0,int(host_memAdd_str,2)) 
        if RVCore == 0:
            RB0Dat.append(hex(datToHost.read()))
        elif RVCore == 1:
            RB1Dat.append(hex(datToHost.read()))
        regBankStr = "X"+"{:02d}".format(i)
        if verbose_prints:
            # if(hex(datToHost.read()) != "0x0"):
            print(regBankStr, '  ', ABI_Array[i], '\tData: ',hex(datToHost.read()))
        elif verbose_prints:
            print(regBankStr, '  ', ABI_Array[i], '\tData: ',hex(datToHost.read()))
        sleep(period)
    hostCtrl.write(0,int('0',2))
    host_memAdd_15 = int('0',2)
    host_memAdd_4_0 = int('00000',2)
    host_memAdd_str = '{0:016b}'.format(host_memAdd_31_16) + '{0:01b}'.format(host_memAdd_15) + '{0:02b}'.format(host_memAdd_14_13) + '{0:08b}'.format(host_memAdd_12_5) + '{0:05b}'.format(host_memAdd_4_0)
    host_memAdd.write(0,int(host_memAdd_str,2))
    sleep(10*period)
    
print("\nOperation Finished")    

Register Bank of RVCore0

X00    zero 	Data:  0x0
X01    ra 	Data:  0x0
X02    sp 	Data:  0x0
X03    gp 	Data:  0x0
X04    tp 	Data:  0x0
X05    t0 	Data:  0x0
X06    t1 	Data:  0x0
X07    t2 	Data:  0x0
X08    s0/fp 	Data:  0x0
X09    s1 	Data:  0x0
X10    a0 	Data:  0x0
X11    a1 	Data:  0x0
X12    a2 	Data:  0x0
X13    a3 	Data:  0x0
X14    a4 	Data:  0x0
X15    a5 	Data:  0x0
X16    a6 	Data:  0x0
X17    a7 	Data:  0x0
X18    s2 	Data:  0x0
X19    s3 	Data:  0x0
X20    s4 	Data:  0x0
X21    s5 	Data:  0x0
X22    s6 	Data:  0x0
X23    s7 	Data:  0x0
X24    s8 	Data:  0x0
X25    s9 	Data:  0x0
X26    s10 	Data:  0x0
X27    s11 	Data:  0x0
X28    t3 	Data:  0x0
X29    t4 	Data:  0x0
X30    t5 	Data:  0x0
X31    t6 	Data:  0x0

Register Bank of RVCore1

X00    zero 	Data:  0x0
X01    ra 	Data:  0x0
X02    sp 	Data:  0x0
X03    gp 	Data:  0x0
X04    tp 	Data:  0x0
X05    t0 	Data:  0x0
X06    t1 	Data:  0x0
X07    t2 	Data:  0x0
X08    s0/fp 	Data:  0x0
X09    s1 	Data:  0x0
X10    a0 	

### Read Machine Code from Files

In [7]:
IM0Array = []; IM1Array = []
for RVCore in range(0,2):
    string = ''; list = []; fullString = ''; data = []; strArray = ''; row = 0
    instrArray = []
    if RVCore == 0:
        file = open(IM0_filename)
        string = file.read(); file.close
        instruction = string.split('\n')
    elif RVCore == 1:
        file = open(IM1_filename)
        string = file.read(); file.close
        instruction = string.split('\n')
    for i in range(0,len(instruction)):
        if (len(instruction[i]) == 10):
            fullString = fullString + instruction[i] + '\n'
        elif(len(instruction[i]) == 8):
            fullString = fullString + "0x" + instruction[i] + '\n'
    for i in range(0,(64-len(instruction))):
            fullString = fullString + '0x00000000' + '\n'
    data = fullString.split('\n')
    for indx in range(0,len(data)-1):
        if indx == 63:
            strArray = strArray + data[indx]
        else:
            strArray = strArray + data[indx] + ", "
        row = row + 1
        if row == 8:
            if indx == 63:
                strArray = strArray
            else:
                strArray = strArray + "\n"
            row = 0     
    if verbose_prints:
        IMArrayStr = 'IM' + str(RVCore) + 'Array = [' + strArray + ']'
        print(IMArrayStr, '\n')
        
    instrArray = strArray.split(", ")
    for i in instrArray:
        if RVCore == 0:
            IM0Array.append(int(i,16))
        elif RVCore == 1:
            IM1Array.append(int(i,16))
print("\nOperation Finished")

IM0Array = [0x0d8000ef, 0x014000ef, 0x034000ef, 0x054000ef, 0x00000013, 0xffdff06f, 0x00f00393, 0x00000413, 
0x00440863, 0x00439393, 0x00140413, 0xfe441ce3, 0x0033f3b3, 0x0033c1b3, 0x00008067, 0x00f00493, 
0x00000513, 0x00650863, 0x00449493, 0x00150513, 0xfe651ce3, 0x0054f4b3, 0x0054c2b3, 0x00008067, 
0x00620663, 0x00624a63, 0x04625063, 0x0091e1b3, 0x0072e2b3, 0x00008067, 0x404305b3, 0x00000613, 
0x0044d493, 0x00160613, 0xfeb61ce3, 0x0091e1b3, 0x00000613, 0x00439393, 0x00160613, 0xfeb61ce3, 
0x0072e2b3, 0x00008067, 0x406205b3, 0x00000613, 0x00449493, 0x00160613, 0xfeb61ce3, 0x0091e1b3, 
0x00000613, 0x0043d393, 0x00160613, 0xfeb61ce3, 0x0072e2b3, 0x00008067, 0xb1ade1b7, 0x00700213, 
0xb01df2b7, 0x0ce28293, 0x00000313, 0x00008067, 0x00000000, 0x00000000, 0x00000000, 0x00000000] 

IM1Array = [0x00000000, 0x00000000, 0x00000000, 0x00000000, 0x00000000, 0x00000000, 0x00000000, 0x00000000, 
0x00000000, 0x00000000, 0x00000000, 0x00000000, 0x00000000, 0x00000000, 0x00000000, 0x00000000, 
0x000

### Write Instruction Memory

In [8]:
for RVCore in range(0,2):
    hostCtrl.write(0,int('1',2))
    host_memWr.write(0,int('1',2))
    bit = str(RVCore)
    host_memAdd_15 = int(bit,2)
    host_memAdd_14_13 = int('00',2) #binary
    host_memAdd_12_6 = int('0000000',2) #binary
    host_memAdd_5_0 = int('000000',2) #binary
    host_memAdd_str = '{0:016b}'.format(host_memAdd_31_16) + '{0:01b}'.format(host_memAdd_15) + '{0:02b}'.format(host_memAdd_14_13) + '{0:07b}'.format(host_memAdd_12_6) + '{0:06b}'.format(host_memAdd_5_0)
    if RVCore == 0:
        print("Write to IM0(Instruction Memory of RVCore0)\n")
    elif RVCore == 1:
        print("\nWrite to IM1(Instruction Memory of RVCore1)\n")    
    for i in range(0,64):
        host_memAdd_5_0 = int('{0:06b}'.format(i),2)
        host_memAdd_str = '{0:016b}'.format(host_memAdd_31_16) + '{0:01b}'.format(host_memAdd_15) + '{0:02b}'.format(host_memAdd_14_13) + '{0:07b}'.format(host_memAdd_12_6) + '{0:06b}'.format(host_memAdd_5_0)
        host_memAdd.write(0,int(host_memAdd_str,2))
        if RVCore == 0:
            host_datToMem.write(0,IM0Array[i])
            if verbose_prints:
                print("Address: ",hex(int(host_memAdd_str,2)), "\t Host Address: ", hex(i), "\tData: ",hex(IM0Array[i]))
        elif RVCore == 1:
            host_datToMem.write(0,IM1Array[i])
            if verbose_prints:
                print("Address: ",hex(int(host_memAdd_str,2)), "\t Host Address: ", hex(i), "\tData: ",hex(IM1Array[i]))
        sleep(period)
    hostCtrl.write(0,int('0',2))
    host_memWr.write(0,int('0',2))
    host_memAdd_15 = int('0',2) #binary
    host_memAdd_5_0 = int('000000',2) #binary
    sleep(10*period)  
print("\nOperation Finished")

Write to IM0(Instruction Memory of RVCore0)

Address:  0x0 	 Host Address:  0x0 	Data:  0xd8000ef
Address:  0x1 	 Host Address:  0x1 	Data:  0x14000ef
Address:  0x2 	 Host Address:  0x2 	Data:  0x34000ef
Address:  0x3 	 Host Address:  0x3 	Data:  0x54000ef
Address:  0x4 	 Host Address:  0x4 	Data:  0x13
Address:  0x5 	 Host Address:  0x5 	Data:  0xffdff06f
Address:  0x6 	 Host Address:  0x6 	Data:  0xf00393
Address:  0x7 	 Host Address:  0x7 	Data:  0x413
Address:  0x8 	 Host Address:  0x8 	Data:  0x440863
Address:  0x9 	 Host Address:  0x9 	Data:  0x439393
Address:  0xa 	 Host Address:  0xa 	Data:  0x140413
Address:  0xb 	 Host Address:  0xb 	Data:  0xfe441ce3
Address:  0xc 	 Host Address:  0xc 	Data:  0x33f3b3
Address:  0xd 	 Host Address:  0xd 	Data:  0x33c1b3
Address:  0xe 	 Host Address:  0xe 	Data:  0x8067
Address:  0xf 	 Host Address:  0xf 	Data:  0xf00493
Address:  0x10 	 Host Address:  0x10 	Data:  0x513
Address:  0x11 	 Host Address:  0x11 	Data:  0x650863
Address:  0x12 	 Ho

### Write Memory Bank

In [ ]:
# Not fixed yet at all
RVCore = 1
hostCtrl.write(0,int('1',2))
host_memWr.write(0,int('1',2))
bit = str(RVCore)
host_memAdd_15 = int(bit,2) 
host_memAdd_14_13 = int('01',2) 
host_memAdd_12_6 = int('0000000',2) 
host_memAdd_5_0 = int('000000',2) 
host_memAdd_str = '{0:016b}'.format(host_memAdd_31_16) + '{0:01b}'.format(host_memAdd_15) + '{0:02b}'.format(host_memAdd_14_13) + '{0:07b}'.format(host_memAdd_12_6) + '{0:06b}'.format(host_memAdd_5_0)
host_memAdd.write(0,int(host_memAdd_str,2))
if RVCore == 0:
    print("Memory of RVCore0\n")
elif RVCore == 1:
    print("\nMemory of RVCore1\n")
for i in range(0,32):
    host_memAdd_5_0 = int('{0:06b}'.format(i),2)
    host_memAdd_str = '{0:016b}'.format(host_memAdd_31_16) + '{0:01b}'.format(host_memAdd_15) + '{0:02b}'.format(host_memAdd_14_13) + '{0:07b}'.format(host_memAdd_12_6) + '{0:06b}'.format(host_memAdd_5_0)
    host_memAdd.write(0,int(host_memAdd_str,2))
    data = int(1234, 2) # Placeholder for real data
    host_datToMem.write(0,data)
    if verbose_prints:
        print("Host Address: ", hex(i),"\tData: ", hex(data));
    sleep(period)
host_memAdd_15 = int('0',2) #binary
host_memAdd_14_13 = int('00',2) #binary
host_memAdd_5_0 = int('000000',2) #binary
host_memAdd_str = '{0:016b}'.format(host_memAdd_31_16) + '{0:01b}'.format(host_memAdd_15) + '{0:02b}'.format(host_memAdd_14_13) + '{0:07b}'.format(host_memAdd_12_6) + '{0:06b}'.format(host_memAdd_5_0)
host_memAdd.write(0,int(host_memAdd_str,2))
sleep(10*period)    
print("\nOperation Finished")

### Run Steps

In [17]:
hostCtrl.write(0,int('0',2))
host_memWr.write(0,int('0',2))
for i in range(0,number_of_steps):
    step.write(0,int('1',2))
    sleep(period)
    step.write(0,int('0',2))
    sleep(period)
print("\nOperation Finished")


Operation Finished


In [10]:
for RVCore in range(0,2):
    hostCtrl.write(0,int('1',2));
    host_memWr.write(0,int('1',2)); 
    bit = str(RVCore)
    host_memAdd_15 = int(bit,2)
    host_memAdd_14_13 = int('01',2) #binary
    host_memAdd_12_6 = int('0000000',2) #binary
    host_memAdd_5_0 = int('000000',2) #binary
    host_memAdd_str = '{0:016b}'.format(host_memAdd_31_16) + '{0:01b}'.format(host_memAdd_15) + '{0:02b}'.format(host_memAdd_14_13) + '{0:07b}'.format(host_memAdd_12_6) + '{0:06b}'.format(host_memAdd_5_0)
    if RVCore == 0:
        print("Memory of RVCore0\n")
    elif RVCore == 1:
        print("\nMemory of RVCore1\n")
    for i in range(0,64):
        host_memAdd_5_0 = int('{0:06b}'.format(i),2)
        host_memAdd_str = '{0:016b}'.format(host_memAdd_31_16) + '{0:01b}'.format(host_memAdd_15) + '{0:02b}'.format(host_memAdd_14_13) + '{0:07b}'.format(host_memAdd_12_6) + '{0:06b}'.format(host_memAdd_5_0)
        host_memAdd.write(0,int(host_memAdd_str,2))
        host_datToMem.write(0,0x00000000)
        if verbose_prints:
            print("Host Address:", hex(i), "\tData: 0x00000000")
        sleep(period)
    
    hostCtrl.write(0,int('0',2))
    host_memWr.write(0,int('0',2))
    host_memAdd_15 = int('0',2)
    host_memAdd_14_13 = int('00',2) 
    host_memAdd_5_0 = int('000000',2) #binary
    host_memAdd_str = '{0:016b}'.format(host_memAdd_31_16) + '{0:01b}'.format(host_memAdd_15) + '{0:02b}'.format(host_memAdd_14_13) + '{0:07b}'.format(host_memAdd_12_6) + '{0:06b}'.format(host_memAdd_5_0)
print("\nOperation Finished")

Memory of RVCore0

Host Address: 0x0 	Data: 0x00000000
Host Address: 0x1 	Data: 0x00000000
Host Address: 0x2 	Data: 0x00000000
Host Address: 0x3 	Data: 0x00000000
Host Address: 0x4 	Data: 0x00000000
Host Address: 0x5 	Data: 0x00000000
Host Address: 0x6 	Data: 0x00000000
Host Address: 0x7 	Data: 0x00000000
Host Address: 0x8 	Data: 0x00000000
Host Address: 0x9 	Data: 0x00000000
Host Address: 0xa 	Data: 0x00000000
Host Address: 0xb 	Data: 0x00000000
Host Address: 0xc 	Data: 0x00000000
Host Address: 0xd 	Data: 0x00000000
Host Address: 0xe 	Data: 0x00000000
Host Address: 0xf 	Data: 0x00000000
Host Address: 0x10 	Data: 0x00000000
Host Address: 0x11 	Data: 0x00000000
Host Address: 0x12 	Data: 0x00000000
Host Address: 0x13 	Data: 0x00000000
Host Address: 0x14 	Data: 0x00000000
Host Address: 0x15 	Data: 0x00000000
Host Address: 0x16 	Data: 0x00000000
Host Address: 0x17 	Data: 0x00000000
Host Address: 0x18 	Data: 0x00000000
Host Address: 0x19 	Data: 0x00000000
Host Address: 0x1a 	Data: 0x0000000